In [1]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date
import json
import re


with open("login.json", "r") as j:
    misDatos = json.load(j)
#para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
#para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [2]:
results = sf.query_all(
    """
    select Id, IdentificationId__pc from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
    }
)
Pacientes_SF.shape



In [ ]:
results = sf.query_all(
    """
    select id,IdentificationId__c from Contact 
   """
)
Pacientes_SF2 = pd.DataFrame(results["records"])

Pacientes_SF2 = Pacientes_SF2[
    [
        "Id",
        "IdentificationId__c",
    ]
]
Pacientes_SF2 = Pacientes_SF2.rename(
    columns={
        "Id": "Id Contact Paciente",
        "IdentificationId__c": "Rut",
    }
)
Pacientes_SF2.shape

In [ ]:
Pacientes_SF2 = pd.merge(
    left=Pacientes_SF,
    right=Pacientes_SF2,
    how="left",
    left_on="Rut",
    right_on="Rut",
)
Pacientes_SF2.shape

In [ ]:
Pacientes_SF2.head()

In [ ]:
results = sf.query_all(
    """
    select id,Paciente__c,Programas_por_Empresas__c from Paciente_Programas__c where Programas_por_Empresas__r.Empresa__r.Estado__c <> 'Dado De Baja - Sin Servicio' 
   """
)
Paciente_Programa = pd.DataFrame(results["records"])

Paciente_Programa = Paciente_Programa[
    [
        "Paciente__c",
        "Programas_por_Empresas__c",
    ]
]
Paciente_Programa = Paciente_Programa.rename(
    columns={
        "Paciente__c": "Id paciente account",
        "Programas_por_Empresas__c": "Programas por empresa",
    }
)
Paciente_Programa.shape

In [ ]:
Paciente_Programa.head()

In [ ]:
results = sf.query_all(
    """
    select Id,Empresa__c from Programas_por_Empresas__c  
   """
)
empresas = pd.DataFrame(results["records"])

empresas = empresas[
    [
        "Id",
        "Empresa__c",
    ]
]
empresas = empresas.rename(
    columns={
        "Id": "Programas_por_Empresas__c",
        "Empresa__c": "Id Account Empresa",
    }
)
empresas.shape





In [ ]:
empresas.shape

In [ ]:
Paciente_Programa = pd.merge(
    left=Paciente_Programa,
    right=empresas,
    how="left",
    left_on="Programas por empresa",
    right_on="Programas_por_Empresas__c",
)
Paciente_Programa.shape

In [ ]:
Paciente_Programa.head()

In [ ]:
results = sf.query_all(
    """
    select AccountId, ContactId from AccountContactRelation 
   """
)
accountcontact = pd.DataFrame(results["records"])

accountcontact = accountcontact[
    [
        "AccountId",
        "ContactId",
    ]
]
accountcontact = accountcontact.rename(
    columns={
        "AccountId": "Id account",
        "ContactId": "id contacto",
    }
)
accountcontact.shape

In [ ]:
accountcontact['LLave'] = accountcontact['id contacto'] + accountcontact['Id account'] 
accountcontact.head()

In [ ]:
accountcontact.shape

In [ ]:
Paciente_Programa.head()

In [ ]:
Paciente_Programa.shape

In [ ]:
Pacientes_SF2.head()

In [ ]:
Pacientes_SF2.shape

In [ ]:
programa_paciente = pd.merge(
    left=Paciente_Programa,
    right=Pacientes_SF2,
    how="left",
    left_on="Id paciente account",
    right_on="Id Account Paciente",
)
programa_paciente.shape

In [ ]:
programa_paciente.head()

In [ ]:
programa_paciente.shape

In [ ]:
programa_paciente = programa_paciente[
    [
        "Id Contact Paciente",
        "Id Account Empresa",
    ]
]
programa_paciente['LLave'] = programa_paciente['Id Contact Paciente'] + programa_paciente['Id Account Empresa']
programa_paciente = programa_paciente.rename(
    columns={
        "Id Contact Paciente": "ContactId",
        "Id Account Empresa": "AccountId",
    }
)
programa_paciente.head()

In [ ]:
programa_paciente.shape

In [ ]:
programa_paciente["RutEstaoNo"] = programa_paciente["LLave"].isin(accountcontact['LLave'])
programa_paciente.shape

In [ ]:
programa_paciente.head()


In [ ]:
programa_paciente1 = programa_paciente[programa_paciente["RutEstaoNo"] == False] 

In [ ]:
accountcontact["RutEstaoNo"] = accountcontact["LLave"].isin(programa_paciente['LLave'])
accountcontact1 = accountcontact[accountcontact["RutEstaoNo"] == False]
accountcontact1.head()


In [ ]:
accountcontact.shape

In [ ]:
programa_paciente1 = programa_paciente1.drop_duplicates(['LLave'], keep='last')
programa_paciente1.shape

In [ ]:
programa_paciente1.to_excel("F:/Empresas SF/reportes/accountcontact.xlsx", index=False)
#accountcontact1.to_excel("F:/Empresas SF/reportes/accountcontact1.xlsx", index=False)